In [37]:
import pandas as pd
import yfinance as yf
import re
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import time

In [39]:
def initializeUnderlyingDataAndDicts():
    global underlyingData, underlyingPriceDict, underlying15DVolDict, underlying30DVolDict, underlying60DVolDict, underlyingYZVolDict

    # Opening underlying file
    underlyingData = pd.read_csv("data/AAPL.csv")
    
    # Prepare all the underlying volatility dicts
    underlyingData["PriceChange"] = underlyingData["Close"] - underlyingData["Close"].shift(-1)
    underlyingData["PercentChange"] = underlyingData["PriceChange"] / (underlyingData["Close"] + underlyingData["PriceChange"])
    underlyingData['15DayVol'] = underlyingData["PercentChange"].rolling(15).std() * (252 ** 0.5)
    underlyingData['30DayVol'] = underlyingData["PercentChange"].rolling(30).std() * (252 ** 0.5)
    underlyingData['60DayVol'] = underlyingData["PercentChange"].rolling(60).std() * (252 ** 0.5)
    
    # Code from here: https://harbourfronts.com/garman-klass-yang-zhang-historical-volatility-calculation-volatility-analysis-python/
    underlyingData['YZVol'] = np.sqrt(252 / 30 * pd.DataFrame.rolling(np.log(underlyingData.loc[:, "Open"] / underlyingData.loc[:, "Close"].shift(1)) ** 2 +
                         0.5 * np.log(underlyingData.loc[:, "High"] / underlyingData.loc[:, "Low"]) ** 2 -
                         (2 * np.log(2) - 1) *
                         np.log(underlyingData.loc[:, "Close"] / underlyingData.loc[:, "Open"]) ** 2,
                         window=30).sum())
    
    # Remove NaN
    underlyingData = underlyingData.dropna()
    underlyingData.reset_index(drop=True)
    
    # Populate global dictionaries
    underlyingPriceDict = dict(zip(underlyingData.Date, underlyingData.Close))
    underlying15DVolDict = dict(zip(underlyingData.Date, underlyingData["15DayVol"]))
    underlying30DVolDict = dict(zip(underlyingData.Date, underlyingData["30DayVol"]))
    underlying60DVolDict = dict(zip(underlyingData.Date, underlyingData["60DayVol"]))
    underlyingYZVolDict = dict(zip(underlyingData.Date, underlyingData.YZVol))



def getObjectiveClosePrice(underlyingData, date):
    # Gets the close price on a date given objective data. None if date DNE
    # Consider using a hash map.
    try:
        return float(underlyingData.loc[underlyingData['Date'] == date]['Close'])
    except:
        return None

def getOptionProfit(expireDate, strike):
    # Gets the profit of an option given its expiration date and strike price
    endPrice = underlyingPriceDict[expireDate]
    if endPrice == None:
        return 0
    return max(endPrice - strike, 0)    
    
def getAndProcessOneYearOptionData():
    initializeUnderlyingDataAndDicts()
    finalDay = '2022-04-27'
    
    # Creating masterDf 
    masterDf = pd.read_csv("data/aapl_eod_202101.txt")
    for i in range(2, 13):
        masterDf = masterDf.append(pd.read_csv("data/aapl_eod_2021" + str(i).zfill(2) + ".txt"))
    
    # Dropping irrelevant columns
    cols_to_drop = [0, 1, 3, 6, 14, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
    masterDf.drop(masterDf.columns[cols_to_drop],axis=1,inplace=True)
    
    # Modifying column names for readability
    col_names = []
    for x in list(masterDf.columns):
        x = x.replace(' [', '').replace(']', '')
        col_names.append(x)

    masterDf.columns = col_names

    # Converting all but a few columns to numeric and removing NaN
    for i in range(len(col_names)):
        if i == 0 or i == 2 or i == 11:
            continue
        masterDf[col_names[i]] = pd.to_numeric(masterDf[col_names[i]], errors='coerce')
        masterDf = masterDf[masterDf[col_names[i]].notna()]
    
    # Stripping strings in problematic columns
    masterDf['QUOTE_DATE'] = masterDf['QUOTE_DATE'].str.strip()
    masterDf['C_SIZE'] = masterDf['C_SIZE'].str.strip()
    masterDf['EXPIRE_DATE'] = masterDf['EXPIRE_DATE'].str.strip()

    # Risk free rate column
    riskFreeRateDf = pd.read_csv("data/RiskFreeRate.csv")
    riskFreeRateDict = dict(zip(riskFreeRateDf.DATE, riskFreeRateDf.DTB3))
    masterDf['RISK_FREE_RATE'] = masterDf['QUOTE_DATE'].map(riskFreeRateDict)
    masterDf['RISK_FREE_RATE'] = pd.to_numeric(masterDf['RISK_FREE_RATE'], errors='coerce')
    masterDf = masterDf[masterDf['RISK_FREE_RATE'].notna()]
    
    # Map volatility columns
    masterDf["15DayVol"] = masterDf["QUOTE_DATE"].map(underlying15DVolDict)
    masterDf["30DayVol"] = masterDf["QUOTE_DATE"].map(underlying30DVolDict)
    masterDf["60DayVol"] = masterDf["QUOTE_DATE"].map(underlying60DVolDict)
    masterDf["YZVol"] = masterDf["QUOTE_DATE"].map(underlyingYZVolDict)
    
    # Converting date columns to dates. Requires .date() to become DateTime
    masterDf['EXPIRE_DATE'] = pd.to_datetime(masterDf['EXPIRE_DATE'])
    masterDf['QUOTE_DATE'] = pd.to_datetime(masterDf['QUOTE_DATE'])
    
    # Remove all options past the last date of underlying data
    masterDf = masterDf[~(masterDf['EXPIRE_DATE'] > finalDay)]
    
    # Map option profits
    masterDf['OPTION_PROFIT'] = masterDf.apply(lambda x: getOptionProfit(str(x['EXPIRE_DATE'].date()), x['STRIKE']), axis=1)
    masterDf['BID_ASK_MIDPOINT'] = masterDf.apply(lambda x: (x['C_ASK'] + x['C_BID']) / 2, axis=1)
    return masterDf

def filter_for_moneyness_and_time(df, low_money = None, high_money = None, low_time = None, high_time = None):
    if low_money == None and high_money == None:
        pass
    elif low_money == None:
        df = df[df['STRIKE'] / df['UNDERLYING_LAST'] <= high_money]
    elif high_money == None:
        df = df[df['STRIKE'] / df['UNDERLYING_LAST'] >= low_money]
    else:
        df = df[(df['STRIKE'] / df['UNDERLYING_LAST'] >= low_money) & (df['STRIKE'] / df['UNDERLYING_LAST'] <= high_money)]

    if low_time == None and high_time == None:
        return df
    if low_time == None:
        return df[df['DTE'] <= high_time]
    elif high_time == None:
        return df[df['DTE'] >= low_time]
    else:
        return df[(df['DTE'] >= low_time) & (df['DTE'] <= high_time)]     
    

# For NN, features are: DTE, Underlying last, Delta, gamma, vega, theta, rho, IB

In [40]:
df = getAndProcessOneYearOptionData()
df

C:\Users\David\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,QUOTE_DATE,UNDERLYING_LAST,EXPIRE_DATE,DTE,C_DELTA,C_GAMMA,C_VEGA,C_THETA,C_RHO,C_IV,...,STRIKE,STRIKE_DISTANCE,STRIKE_DISTANCE_PCT,RISK_FREE_RATE,15DayVol,30DayVol,60DayVol,YZVol,OPTION_PROFIT,BID_ASK_MIDPOINT
13,2021-01-04,129.45,2021-01-08,4.00,0.99330,0.00182,0.00267,-0.01615,0.01475,0.70653,...,106.0,23.4,0.181,0.09,0.335185,0.290138,0.345166,0.282196,26.050003,23.475
16,2021-01-04,129.45,2021-01-08,4.00,0.99171,0.00236,0.00355,-0.01585,0.01479,0.61801,...,109.0,20.4,0.158,0.09,0.335185,0.290138,0.345166,0.282196,23.050003,20.475
17,2021-01-04,129.45,2021-01-08,4.00,0.99211,0.00254,0.00301,-0.01615,0.01530,0.58957,...,110.0,19.4,0.150,0.09,0.335185,0.290138,0.345166,0.282196,22.050003,19.480
18,2021-01-04,129.45,2021-01-08,4.00,0.98019,0.00493,0.00688,-0.04014,0.01468,0.65222,...,111.0,18.4,0.143,0.09,0.335185,0.290138,0.345166,0.282196,21.050003,18.520
19,2021-01-04,129.45,2021-01-08,4.00,0.98415,0.00440,0.00543,-0.02860,0.01464,0.58313,...,112.0,17.4,0.135,0.09,0.335185,0.290138,0.345166,0.282196,20.050003,17.495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19210,2021-12-31,177.58,2022-04-14,103.96,0.06265,0.00433,0.11726,-0.01649,0.02991,0.29953,...,230.0,52.4,0.295,0.06,0.293058,0.297586,0.243602,0.311005,0.000000,0.730
19211,2021-12-31,177.58,2022-04-14,103.96,0.05137,0.00364,0.10102,-0.01461,0.02492,0.30623,...,235.0,57.4,0.323,0.06,0.293058,0.297586,0.243602,0.311005,0.000000,0.595
19212,2021-12-31,177.58,2022-04-14,103.96,0.04452,0.00317,0.08905,-0.01318,0.02085,0.31495,...,240.0,62.4,0.351,0.06,0.293058,0.297586,0.243602,0.311005,0.000000,0.515
19213,2021-12-31,177.58,2022-04-14,103.96,0.03869,0.00268,0.08006,-0.01283,0.01835,0.32534,...,245.0,67.4,0.380,0.06,0.293058,0.297586,0.243602,0.311005,0.000000,0.455


In [51]:
def train_and_find_rmse(low_money, high_money, low_time, high_time, df):
    start = time.time()
    df = filter_for_moneyness_and_time(df, low_money, high_money, low_time, high_time)
    X = df.iloc[:, [1, 3, 14, 17, 18, 19, 20, 21]].values
    y = df.iloc[:, -1].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 0)
    
    sc = MinMaxScaler()
    X_train_sc = pd.DataFrame(sc.fit_transform(X_train))
    X_test_sc = pd.DataFrame(sc.transform(X_test))
    
    model = MLPRegressor(activation = 'relu', max_iter = 1000, tol = 0.0001)
    model.fit(X_train_sc, y_train)
    
    y_train_pred = model.predict(X_train_sc)
    y_test_pred = model.predict(X_test_sc)
    print("RMSE on test data for moneyness(" + str(low_money) + ", " + str(high_money) + ") and DTE(" + str(low_time) + ", " + 
          str(high_time) + "): " + str(metrics.mean_squared_error(y_test, y_test_pred) ** 0.5))
    print("Run in: " + str(time.time() - start) + " sec")
    return model

In [52]:
train_and_find_rmse(0.2, 0.8, 0, 30, df)
train_and_find_rmse(0.8, 1.2, 0, 30, df)
train_and_find_rmse(1.2, 1.8, 0, 30, df)

train_and_find_rmse(0.2, 0.8, 30, 90, df)
train_and_find_rmse(0.8, 1.2, 30, 90, df)
train_and_find_rmse(1.2, 1.8, 30, 90, df)

train_and_find_rmse(0.2, 0.8, 90, 300, df)
train_and_find_rmse(0.8, 1.2, 90, 300, df)
train_and_find_rmse(1.2, 1.8, 90, 300, df)

RMSE on test data for moneyness(0.2, 0.8) and DTE(0, 30): 0.16493841838781506
Run in: 6.522980213165283 sec
RMSE on test data for moneyness(0.8, 1.2) and DTE(0, 30): 0.2915803300342415
Run in: 31.825873374938965 sec
RMSE on test data for moneyness(1.2, 1.8) and DTE(0, 30): 0.05598869879874404
Run in: 0.5089442729949951 sec
RMSE on test data for moneyness(0.2, 0.8) and DTE(30, 90): 0.20506430534343129
Run in: 8.129276990890503 sec
RMSE on test data for moneyness(0.8, 1.2) and DTE(30, 90): 0.3073986492985146
Run in: 27.240172147750854 sec
RMSE on test data for moneyness(1.2, 1.8) and DTE(30, 90): 0.10839112805096184
Run in: 2.3421435356140137 sec
RMSE on test data for moneyness(0.2, 0.8) and DTE(90, 300): 0.4150076574554223
Run in: 9.2562837600708 sec
RMSE on test data for moneyness(0.8, 1.2) and DTE(90, 300): 0.33663777868971007
Run in: 21.532235145568848 sec
RMSE on test data for moneyness(1.2, 1.8) and DTE(90, 300): 0.16217770402800136
Run in: 5.257808685302734 sec


MLPRegressor(max_iter=1000)

In [9]:
df.columns[18]

'15DayVol'

In [10]:
X = df.iloc[:, [1, 3, 14, 17, 18, 19, 20, 21]].values

y = df.iloc[:, 13].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [12]:
X_a = df.iloc[:, [1, 3, 14, 17, 18, 19, 20, 21]]
X_a

,UNDERLYING_LAST,DTE,STRIKE,RISK_FREE_RATE,15DayVol,30DayVol,60DayVol,YZVol
13,129.45,4.00,106.0,0.09,0.335185,0.290138,0.345166,0.282196
16,129.45,4.00,109.0,0.09,0.335185,0.290138,0.345166,0.282196
17,129.45,4.00,110.0,0.09,0.335185,0.290138,0.345166,0.282196
18,129.45,4.00,111.0,0.09,0.335185,0.290138,0.345166,0.282196
19,129.45,4.00,112.0,0.09,0.335185,0.290138,0.345166,0.282196
...,...,...,...,...,...,...,...,...
19210,177.58,103.96,230.0,0.06,0.293058,0.297586,0.243602,0.311005
19211,177.58,103.96,235.0,0.06,0.293058,0.297586,0.243602,0.311005
19212,177.58,103.96,240.0,0.06,0.293058,0.297586,0.243602,0.311005
19213,177.58,103.96,245.0,0.06,0.293058,0.297586,0.243602,0.311005


In [13]:
sc = MinMaxScaler()
X_train_sc = pd.DataFrame(sc.fit_transform(X_train))
X_test_sc = pd.DataFrame(sc.transform(X_test))

In [31]:
model = MLPRegressor(verbose = True, activation = 'relu', tol = 0.0005)
model.fit(X_train_sc, y_train)


# YZ 100 iterations loss = 11.41, relu
# YZ 100 iterations loss = 13.9, logistic

Iteration 1, loss = 473.04730952
Iteration 2, loss = 207.11050260
Iteration 3, loss = 89.00879599
Iteration 4, loss = 57.62492112
Iteration 5, loss = 37.82534953
Iteration 6, loss = 25.04113368
Iteration 7, loss = 15.08562125
Iteration 8, loss = 8.19461064
Iteration 9, loss = 4.39124515
Iteration 10, loss = 2.43297173
Iteration 11, loss = 1.39164803
Iteration 12, loss = 0.83796435
Iteration 13, loss = 0.53785984
Iteration 14, loss = 0.36899548
Iteration 15, loss = 0.27389187
Iteration 16, loss = 0.21895578
Iteration 17, loss = 0.18750380
Iteration 18, loss = 0.16883576
Iteration 19, loss = 0.15785073
Iteration 20, loss = 0.15210131
Iteration 21, loss = 0.14851065
Iteration 22, loss = 0.14444459
Iteration 23, loss = 0.14257943
Iteration 24, loss = 0.14144144
Iteration 25, loss = 0.13944022
Iteration 26, loss = 0.13769351
Iteration 27, loss = 0.13746958
Iteration 28, loss = 0.13571051
Iteration 29, loss = 0.13438439
Iteration 30, loss = 0.13299452
Iteration 31, loss = 0.13236118
Iteratio

MLPRegressor(tol=0.0005, verbose=True)

In [32]:
y_train_pred = model.predict(X_train_sc)
y_test_pred = model.predict(X_test_sc)
print(metrics.mean_squared_error(y_train, y_train_pred))
print(metrics.mean_squared_error(y_test, y_test_pred))

0.18098016048108098
0.18616766033295393


In [21]:
max(y_test)

71.13999899999999

In [20]:
max(y_test_pred)

63.02642172898726